In [9]:
# Genetic Algorithm Class for hyperparamter optimisation
#          generate_inital_popuilation()
#          select()
#          mate()
#          mutate()
#          evaluate()

import random
import numpy as np
import kf_ml_lib as kf

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

class kf_genetic_algorithm:
    def __init__(self, classifier, dataset_path, population_size, generations):
        self.classifier = classifier
        self.population_size = population_size
        self.generations = generations
        
        # Gene pool
        # Set options and constraints for the selected classifier's hyperparameters
        if self.classifier == "DecisionTreeClassifier":
            self.criterions = ['gini', 'entropy']
            self.splits = ['best', 'random']
            self.min_samples_splits = dict(low = 2, high = 5)
            self.min_samples_leafs = dict(low = 0.0, high = 0.5)
            self.min_weight_fraction_leafs = dict(low = 0, high = 0.5)
            self.class_weights = ['balanced', None]
                
        # Generate inital chromosome population, with size population_size
        self.population = self.generate_inital_population()
        
        # Load, process and split dataset
        self.dataset = kf.load_dataset(dataset_path)
        self.X, self.y = kf.split_dataset(self.dataset, extended=False)

        self.y = np.split(self.y, 1)
        self.y = np.array(self.y)
        self.y = self.y.reshape(len(self.y[0]), 1)

        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, train_size=0.7, stratify=self.y)
        
    
    def __del__(self):
        print("\nDestroying ga object\n")

        
    # Function to generate an inital random population of chromosomes, given a population_size
    def generate_inital_population(self):
        print("Generating Population of size: ", self.population_size)
        
        # Inital chromosome population
        population = []
        
        # Fill each chromosome with randomly selected genes from the gene pools and add it to the population
        for i in range(0, self.population_size):
            chromosome = []
            chromosome.append(random.choice(self.criterions))
            chromosome.append(random.choice(self.splits))
            chromosome.append(random.randrange(self.min_samples_splits['low'], self.min_samples_splits['high']))
            chromosome.append(random.uniform(self.min_samples_leafs['low'], self.min_samples_leafs['high']))
            chromosome.append(random.uniform(self.min_weight_fraction_leafs['low'], self.min_weight_fraction_leafs['high']))   
            chromosome.append(random.choice(self.class_weights))
            
            population.append(chromosome)
        
        return population

    
    # Run the Genetic Algorithm for N generations
    def run(self):        
        #for generation in range(0, generations):
        
        # Generate fitness scores for all the chromosomes in the population
        fitness_scores = self.evaluate()           

        # Select the best two performing chromosomes to be parents
        parent1, parent2 = self.selection(fitness_scores)

        # Mate parents using one-point-crossover in order to generate offspring
        offspring1, offspring2 = self.crossover(parent1, parent2)
        
        # Mutate the offspring, selected at random, into the remaining population spaces
        # REMAINING POPULATION SPACES!!! IMPORTANT
        # use something like:
        #        self.population = []
        #        self.population = [parent1, parent2, offspring1, offspring2]
        #        remaining_pop_space = self.population_size - len(self.population)
        #
        #        mutate(remaining_pop_space)
        # 
        #        def mutate(remaining_pop_space):
        #            for chromosome in remaining_pop_space:
        #                  chromosome = self.population[3]
        #                  
        #                   # select random int to index a gene to mutate
        #                   # then must mutate depending upon the chromosome's gene index, relating to the gene pool
        #
        #                   append chromosome to the self.population

        # return best_result
        
    
    # Generate fitness_scores for the given population
    def evaluate(self):
        fitness_scores = []
        
        for chromosome in self.population:
            dtc = DecisionTreeClassifier(criterion=chromosome[0],
                                         splitter=chromosome[1],
                                         min_samples_split=chromosome[2],
                                         min_samples_leaf=chromosome[3],
                                         min_weight_fraction_leaf=chromosome[4],
                                         class_weight=chromosome[5])
            
            dtc = dtc.fit(self.X_train, self.y_train)
            
            # Make predictions on test data for this chromosome
            predictions = dtc.predict(self.X_test)
            
            # Evaluate model Precision, Recall and F1-Score performance
            precision = metrics.precision_score(self.y_test, predictions, pos_label='Botnet')
            recall = metrics.recall_score(self.y_test, predictions, pos_label='Botnet')
            f1_score = kf.calc_f1_score(precision, recall)

            fitness_scores.append(f1_score)
    
        return fitness_scores
    
    # Perform tournament selection on the evaluated chromosomes; returning the two best performers
    def selection(self, fitness_scores):        
        # Sort, from highest to lowest, the fitness_scores and their chromosomes
        sorted_results = sorted(zip(fitness_scores, self.population), key=lambda x: x[0], reverse=True)
        
        # Remove 'nan' fitness_value results from the sorted_results
        for result in sorted_results:
            if np.isnan(result[0]):
                sorted_results.remove(result)
        
        print(sorted_results)
        
        parent1 = sorted_results[0]
        parent2 = sorted_results[1]
        
        print("Parent 1 = ", parent1)
        print("Parent 2 = ", parent2)
        
        # KYLE-_A
        # REMOVE FITNESS SCORES FROM THE CHROMOSOMES I.E UNZIP THEM FROM THE FITNESSS SCORES
        
        return parent1, parent2
        
    # Performs one-point-crossover of the parents, returning 
    def crossover(self, parent1, parent2):
        
        # KYLE TODO
        #     REMOVE THE FITNESS SCORE FROM PARENT1 AND PARENT2 
        #     IN THE SELECTION() FUNCTION BEFORE THEY ARE RETURNED!!!
        #          SEE 'KYLE-_A'
        
        print("\n\nCrossover:")
        print(parent1)
        print(parent2)
        
        return offspring1, offspring2

In [10]:
# Instantiate Genetic Algoritm class, which generates ga.population of size population_size
# The population's chromosome genes depend upon the classifier being used
# Different classifiers have different 'gene pools' (hyperparameter options)
dataset_path = "../Datasets/ISOT Botnet 2010/Pre-processed/isot_botnet.csv"

ga = kf_genetic_algorithm(classifier='DecisionTreeClassifier', dataset_path=dataset_path, population_size=10, generations=10)

#print("\nInital Population:")
#for chromosome in range(0, ga.population_size):
#    print(ga.population[chromosome])

# Run Genetic Algorithm for N generations on given dataset
best_result = ga.run()


del ga

Generating Population of size:  10


c:\users\asmarus\anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
G:\USB Drive Files\Year 3\Semester 1\KV6003 - Individual Computing Project\Machine Learning\kf_ml_lib.py:91: RuntimeWarning: invalid value encountered in double_scalars
  f1_score = 2 * ((precision*recall) / (precision+recall))
c:\users\asmarus\anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
G:\USB Drive Files\Year 3\Semester 1\KV6003 - Individual Computing Project\Machine Learning\kf_ml_lib.py:91: RuntimeWarning: invalid value e

[(0.6880424580228744, ['gini', 'best', 4, 0.1323130716702775, 0.1511691212720258, 'balanced']), (0.6506608391085034, ['entropy', 'best', 2, 0.3055614891036466, 0.068207406701416, None]), (0.6506608391085034, ['entropy', 'best', 2, 0.2959309501264299, 0.11852828211137273, 'balanced']), (0.6404581334158799, ['entropy', 'best', 3, 0.3483590146938983, 0.15805766259839937, None]), (0.5120032990662465, ['entropy', 'random', 2, 0.43127530609266995, 0.1247044307399302, 'balanced']), (0.5120032990662465, ['entropy', 'random', 4, 0.02330734031308529, 0.38677185655355717, 'balanced']), (0.6506608391085034, ['entropy', 'best', 3, 0.3047742823709143, 0.10011977362152424, None])]
Parent 1 =  (0.6880424580228744, ['gini', 'best', 4, 0.1323130716702775, 0.1511691212720258, 'balanced'])
Parent 2 =  (0.6506608391085034, ['entropy', 'best', 2, 0.3055614891036466, 0.068207406701416, None])


AttributeError: 'kf_genetic_algorithm' object has no attribute 'crossover'